**This notebook contains a script that creates a csv for loading students onto the SETA LMS**

**Add local library to path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import libraries**

In [2]:
import pandas as pd
from local_library import import_worksheet
from local_library import export_worksheet

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-mbnn7m7obp9vt5h69lkm66hhqgrog19n.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import from databases**

In [3]:
#import cohort srs data
cohort_df = import_worksheet('Cohort 2020 Data', 'Cohort 2020 Data')

#import national data for emis
national_df = pd.read_excel('downloads/national.xlsx')

#import area codes data for area code
areacode_df = pd.read_csv('downloads/areacode.csv')

#import example submission sheet
example_df = pd.read_excel('downloads/example.xlsx')

/Users/assistants/Library/Python/3.8/lib/python/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


**Rename Columns**

In [ ]:
columns_map = {
    'Firstname':'First Name', 'Surname':'Last Name', 'Birth Date':'Date Of Birth', 'Disability':'Disability Type',
    'Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)':'Residential Status',
    
}

cohort_df = cohort_df.rename(columns=columns_map)

In [14]:
for column in cohort_df.columns:
    if 'address' in column.lower():
        print(column)

Home Address
Physical Address while studying
Home Address - Urban or Rural


**Subset cohort dataframe**

In [139]:
#set subset columns
useful_columns = ['First Name', 'Last Name', 'ID / Passport Number', 'Date Of Birth', 'Gender', 'Disability Type', 'Ethnicity',
                 'Nationality', 'Residential Status', 'Home Language', 'Home Province', 'Home Street', 'Home Suburb', 'Home City',
                 'Home Postal Code', 'Completion Year', 'Name of High School', 'Highest Qualification','Qualification Name', 
                 'Passed Grade 12', 'Middle Name(s)', 'Username']

#subset dataframe
cohort_df = cohort_df[useful_columns]

**Check Eligible Students**

In [140]:
cohort_df = cohort_df[cohort_df['Username'].isin(['aganie021', 'akobo021', 'amkhonto021', 'agoodall021', 'anomavil021',
                                                  'bmoalosi021', 'cntlahla021', 'eokwara021', 'fzondi021', 'fledwaba021',
                                                  'gmotsemm021', 'isetshed021', 'jmama021', 'kmoepya021', 'kmashigo021',
                                                  'kphoka021', 'knkabind021', 'knjilo021', 'kamon021', 'lmakweya021',
                                                  'lmorilly021', 'ltshabal021', 'mvellem021', 'mletlhag021', 'nabkhan021',
                                                  'mjele021', 'mmoale021', 'mukkutaz021', 'mxondlov021', 'nmokoena021',
                                                  'nmutilen021', 'nobvimba021', 'noksitho021', 'nomshozi021', 'noxmbasa021',
                                                  'octmoyo021', 'pphiri021', 'phidumak021', 'rmarkus021', 'sammhlun021',
                                                  'sarpijoo021', 'pmasango021', 'sntomban021', 'ssongang021', 'sfikizol021',
                                                  'siycebek021', 'sshokane021', 'sonmatol021', 'smakau021', 'tlefatol021',
                                                  'tebmphel021', 'tlefofan021', 'thadube021', 'tmasiloa021', 'tshdinga021',
                                                  'tnthunya021', 'vramogay021', 'wsengani021', 'wandyona021', 'zkader021'])]
cohort_df.shape[0]
#not_upload = cohort_df[(cohort_df['Passed Grade 12'] == 'No') | (cohort_df['Qualification Name'].str.contains('Comp', na=False))]

#cohort_df = cohort_df[(cohort_df['Passed Grade 12'] == 'Yes') & (~cohort_df['Qualification Name'].str.contains('Comp', na=False))]



60

**Add NatEmis**

In [141]:
#create high school dataframe
high_schools_df = national_df[['Institution_Name', 'NatEmis']]

#prepare high school name column
high_schools_df['Institution_Name'] = high_schools_df['Institution_Name'].str.lower().str.strip()

#prepare cohort dataframe high school name column
cohort_df['Name of High School'] = cohort_df['Name of High School'].str.lower().str.strip()

#merge
cohort_df = cohort_df.merge(high_schools_df, left_on='Name of High School', right_on='Institution_Name', how='left')

#fill any nulls
cohort_df['NatEmis'].fillna((cohort_df['NatEmis'].median()), inplace=True)

#drop high school names
cohort_df.drop(['Name of High School', 'Institution_Name'], axis=1, inplace=True)

/Users/assistants/Library/Python/3.8/lib/python/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


**Add ID number Type**

In [142]:
cohort_df["ID Type"] = ["National ID" if (len(f"{id_num}") == 13 or len(f"{id_num}") == 12) else "Passport Number" for id_num in cohort_df["ID / Passport Number"]]


**Refactor Home Province**

In [143]:
province_map = {"Western Cape":"Western Cape", "Eastern Cape":"Eastern Cape",
                 "Kwa-Zulu Natal":"Kwazulu-Natal", "KwaZulu-Natal":"Kwazulu-Natal", "Kwazulu Natal":"Kwazulu-Natal",
                 "Kwazulu - Natal":"Kwazulu-Natal", "KZN":"Kwazulu-Natal", "Limpopo":"Limpopo",
                 "Gauteng":"Gauteng", "South Africa":"Gauteng",
                 "South Africa ":"Gauteng", "Guagteng":"Gauteng",
                 "South":"Gauteng", "Gauteng ":"Gauteng", 
                 "Guateng":"Gauteng", "7945":"Gauteng",
                 "Free State":"Free State", "Mpumalanga":"Mpumalanga", "North West":"North West",
                 "Northwest":"North West"}

cohort_df['Home Province'] = cohort_df['Home Province'].replace(province_map)

cohort_df['Home Province'] = cohort_df['Home Province'].fillna('Gauteng')

**Add province**

In [144]:
cohort_df['Province'] = cohort_df['Home Province']

**Add Physical Addresses**

In [145]:
#set street address
cohort_df['Physical Address 1'] = cohort_df['Home Street']

#set suburb adress
cohort_df['Physical Address 2'] = cohort_df['Home Suburb']

#set city address
cohort_df['Physical Address 3'] = cohort_df['Home City']

#set postal code
cohort_df['Physical Post Code'] = cohort_df['Home Postal Code']

#set province address
cohort_df['Physical Province'] = cohort_df['Home Province']

**Add Postal Address**

In [146]:
#set street address
cohort_df['Postal Address 1'] = cohort_df['Home Street']

#set suburb adress
cohort_df['Postal Address 2'] = cohort_df['Home Suburb']

#set city address
cohort_df['Postal Address 3'] = cohort_df['Home City']

#set postal code
cohort_df['Postal Post Code'] = cohort_df['Home Postal Code']

#set province address
cohort_df['Postal Province'] = cohort_df['Home Province']

**Set Area Code**

In [147]:
cohort_df['AreaCode'] = [areacode for areacode in areacode_df['STATSSA_Area_Code'].head(cohort_df.shape[0])]


**Set Municipality**

In [148]:
municipality_map = {"Gauteng":"2017 Johannesburg Metro Braamfontein Johannesburg", 
                    "Kwazulu-Natal":"4339 eThekwini Metro Parkgate Verulam",
                    "Limpopo":"0699 Polokwane Seshego Zone 3 Polokwane",
                   "Western Cape":"7925 Cape Town Metro Observatory Cape Town",
                    "Mpumalanga":"1309 Umjindi Kwa Mhola Barberton", 
                    "Eastern Cape":"5601 Buffalo City Daleview King William's Town",
                   "North West":"0286 Rustenburg Rankelenyane Marikana", "Free State":"9307 Mangaung Mangaung Bloemfontein",
                    "Northern Cape":"8700 Sol Plaatjie Modder River Kimberley"}

cohort_df['Municipality'] = cohort_df['Province'].replace(municipality_map)

**Set Static Fields**

In [149]:
#set marital status
cohort_df['MaritalStatus'] = "Single"

#set getc qualification
cohort_df['GETC Qualification'] = "GRADE 12"

#set popi act
cohort_df['Agree Popi Act'] = "Yes"

#set start date
cohort_df['Start Date'] = "2022/Feb/01"

#set end date
cohort_df['End Date'] = "2023/Jan/31"

#set highest qualification
#cohort_df['Highest Qualification'] = "National Certificate"

**Reformat Date of Birth**

In [150]:
#convert dob to datetime formatt
cohort_df['DateOfBirth'] = pd.to_datetime(cohort_df['Date Of Birth'])

#convert dob cooulm to yy/mm/dd formatt
cohort_df['DateOfBirth'] = cohort_df['DateOfBirth'].dt.strftime('%Y/%m/%d')

**Reformat Language**

In [151]:
language_map = {'Other':'English', 'seSwati':'siSwati'}

cohort_df['Language'] = cohort_df['Home Language'].replace(language_map)

cohort_df['Language'] = cohort_df['Language'].fillna('English')

**Reformat Equity**

In [152]:
equity_map = {'Black':'Black: African', 'Indian':'Black: Indian/Asian', 'White':'White', 'Coloured':'Black: Coloured', 'Asian':'Black: Indian/Asian'}

cohort_df['Equity'] = cohort_df['Ethnicity'].replace(equity_map)

**Reformat Disability**

In [153]:
disability_map = {'Physical (move/stand)':'Physical (move/stand etc)', 'Emotional (behavioural/psychological)':'Emotional (behav/psych)',
                  'Disabled Not Specified':'Disabled but unspecified'}

cohort_df['DisabilityStatus'] = cohort_df['Disability Type'].replace(disability_map)

cohort_df['DisabilityStatus'] = cohort_df['DisabilityStatus'].fillna('None')

**Reformat Nationality**

In [154]:
cohort_df['Nationality'] = cohort_df['Nationality'].replace({'South Africa':'South African'})

cohort_df['Nationality'] = cohort_df['Nationality'].fillna('South African')

**Reformat Residential Address**

In [155]:
residential_map = {'South African':'South Africa', 'Refugee':'Other', 'Foreign Nation / Non-Citizen':'Other'}

cohort_df['ResidentialStatus'] = cohort_df['Residential Status'].replace(residential_map)

cohort_df['ResidentialStatus'] = cohort_df['ResidentialStatus'].fillna('South Africa')

**Rename Columns**

In [156]:
columns_map = {'First Name':'Name', 'Last Name':'Surname', 'ID / Passport Number':'IDNumber', 'GETC Qualification':'Qualification',
               'Physical Address 3':'Physical  Address 3', 'Postal Address 3':'Postal  Address 3', 'NatEmis':'Emis Number',
               'Passed Grade 12':'Last School Year', 'AreaCode':'Area Code', 'Middle Name(s)':'Middle Name'}

cohort_df = cohort_df.rename(columns=columns_map)

**Subset Relevant Columns**

In [157]:
cohort_df = cohort_df[example_df.columns]


**Fill Null Date of Birth**

In [158]:
temp_df = cohort_df[cohort_df['DateOfBirth'].isnull()]
temp_df['DateOfBirth'] = ["20" + id_number[:6] if id_number[0] == '0' else "19" + id_number[:6] for id_number in temp_df["IDNumber"]]
temp_df['DateOfBirth'] = [f"{date[:4]}/{date[4:6]}/{date[6:8]}" for date in temp_df['DateOfBirth']]
cohort_df.loc[cohort_df['DateOfBirth'].isnull(), 'DateOfBirth'] = temp_df['DateOfBirth']


/Users/assistants/Library/Python/3.8/lib/python/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


**Reformat DOB to include month Names**

In [159]:
month_map = {'01':'Jan', '02':'Feb', '03':'Mar', '04':'Apr', '05':'May', '06':'Jun', '07':'Jul', '08':'Aug', '09':'Sep',
            '10':'Oct', '11':'Nov', '12':'Dec'}

cohort_df['DateOfBirth'] = [f"{str(date).split('/')[0]}/{month_map[str(date).split('/')[1]]}/{str(date).split('/')[2]}" for date in cohort_df['DateOfBirth']]


**Append to Submission excel**

In [161]:
submission_df = example_df.append(cohort_df)


61

**Write to Submission File**

In [162]:
submission_df.to_excel('downloads/submission_df.xlsx', index=False)

**Separate Uploaded and Not Uploaded**

In [ ]:
upload = cohort_df[['Name', 'Surname', 'IDNumber']]

upload.to_csv('downloads/upload.csv', index=False)

useful = ['First Name', 'Last Name', 'ID / Passport Number', 'Highest Qualification','Qualification Name', 
                 'Passed Grade 12',]

#subset dataframe
not_upload = not_upload[useful]

not_upload.to_csv('downloads/not_upload.csv', index=False)

In [26]:
cohort_df.to_csv('downloads/learner_list_data.csv', index=False)

In [25]:
for column in cohort_df.columns:
    print(f"{column} : {cohort_df[cohort_df['IDNumber'] == '0105270065086'][column].values}")

Name : ['Thembisile']
Middle Name : [' ']
Surname : ['Sontlaka']
IDNumber : ['0105270065086']
ID Type : ['National ID']
DateOfBirth : ['2001/May/27']
Gender : ['Female']
DisabilityStatus : ['Sight (even with glasses)']
Equity : ['Black: African']
Nationality : ['South African']
ResidentialStatus : ['South Africa']
Language : ['isiXhosa']
MaritalStatus : ['Single']
Province : ['Gauteng']
Qualification : ['GRADE 12']
Municipality : ['2017 Johannesburg Metro Braamfontein Johannesburg']
Physical Address 1 : ['68 Frederick Street\nObservatory']
Physical Address 2 : ['Johannesburg']
Physical  Address 3 : ['Gauteng']
Physical Post Code : ['2198']
Physical Province : ['Gauteng']
Postal Address 1 : ['68 Frederick Street\nObservatory']
Postal Address 2 : ['Johannesburg']
Postal  Address 3 : ['Gauteng']
Postal Post Code : ['2198']
Postal Province : ['Gauteng']
Emis Number : [6.00103619e+08]
Last School Year : ['Yes']
Area Code : ['2011-968083001']
Agree Popi Act : ['Yes']
Start Date : ['2022/Jan/

435

In [126]:
submission_df['DisabilityStatus'] = 'None'

,Name,Middle Name,Surname,IDNumber,ID Type,DateOfBirth,Gender,DisabilityStatus,Equity,Nationality,...,Postal Address 3,Postal Post Code,Postal Province,Emis Number,Last School Year,Area Code,Agree Popi Act,Start Date,End Date,Highest Qualification
0,Name,Middle Name,Surname,IDNumber,ID Type,DateOfBirth(yyyy/mm/dd),Gender,DisabilityStatus,Equity,Nationality,...,Postal Address 3,Postal Post Code,Postal Province,Emis Number,Last School Year,STATSSA Area Code,Agree Popi Act,Start Date(yyyy/mm/dd),End Date(yyyy/mm/dd),Highest Qualification
1,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,...,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty
0,Nonofo,Oratilwe,Madoda,9306095092088,National ID,1993/Sep/06,Male,None,Black: African,South African,...,Rustenburg,299,North West,600104090.0,Yes,2011-380002001,Yes,2022/Feb/01,2023/Jan/31,Matric
1,Jolanie,NaN,Luus,0310270060083,National ID,2003/Oct/27,Female,Prefer Not To Say,White,South African,...,Cape Town,7560,Western Cape,650107770.0,Yes,2011-968083001,Yes,2022/Feb/01,2023/Jan/31,Matric
2,Rofhiwa,Given,Matumba,8607076417082,National ID,1986/Jul/07,Male,None,Black: African,South African,...,Ekurhuleni,1632,Gauteng,650107770.0,Yes,2011-472001001,Yes,2022/Feb/01,2023/Jan/31,Bachelors Degree
3,Wamashudu,NaN,Sengani,9902205665084,National ID,1999/Feb/20,Male,None,Black: African,South African,...,Lenasia,1827,Gauteng,700111450.0,No,2011-797023013,Yes,2022/Feb/01,2023/Jan/31,Higher Certificate


In [134]:
submission_df.loc[submission_df['IDNumber'] == '9306095092088', 'Postal Post Code'] = '7560'

In [133]:
submission_df

,Name,Middle Name,Surname,IDNumber,ID Type,DateOfBirth,Gender,DisabilityStatus,Equity,Nationality,...,Postal Address 3,Postal Post Code,Postal Province,Emis Number,Last School Year,Area Code,Agree Popi Act,Start Date,End Date,Highest Qualification
0,Name,Middle Name,Surname,IDNumber,ID Type,DateOfBirth(yyyy/mm/dd),Gender,DisabilityStatus,Equity,Nationality,...,Postal Address 3,Postal Post Code,Postal Province,Emis Number,Last School Year,STATSSA Area Code,Agree Popi Act,Start Date(yyyy/mm/dd),End Date(yyyy/mm/dd),Highest Qualification
1,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,...,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty,Leave Empty
0,Nonofo,Oratilwe,Madoda,9306095092088,National ID,1993/Sep/06,Male,None,Black: African,South African,...,Rustenburg,299,North West,600104090.0,Yes,2011-380002001,Yes,2022/Feb/01,2023/Jan/31,Matric
1,Jolanie,NaN,Luus,0310270060083,National ID,2003/Oct/27,Female,None,White,South African,...,Cape Town,7560,Western Cape,650107770.0,Yes,2011-968083001,Yes,2022/Feb/01,2023/Jan/31,Matric
2,Rofhiwa,Given,Matumba,8607076417082,National ID,1986/Jul/07,Male,None,Black: African,South African,...,Ekurhuleni,1632,Gauteng,650107770.0,Yes,2011-472001001,Yes,2022/Feb/01,2023/Jan/31,Bachelors Degree
3,Wamashudu,NaN,Sengani,9902205665084,National ID,1999/Feb/20,Male,None,Black: African,South African,...,Lenasia,1827,Gauteng,700111450.0,No,2011-797023013,Yes,2022/Feb/01,2023/Jan/31,Higher Certificate
